In [26]:
import os
import shutil
import glob
import numpy as np 
import pandas as pd
import pretty_midi
import pypianoroll
# import tables
from music21 import converter, instrument, note, chord, stream
import music21
import librosa
import librosa.display
import matplotlib.pyplot as plt
from keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from tqdm.notebook import tqdm, trange  

import random
import itertools
root_dir = r'C:\Users\ahdz3\Desktop\jazzmidis'
data_dir = root_dir + '\Jazz Midi'
# music_dataset_lpd_dir = root_dir + '/Music Dataset/midis/lmd_matched'

In [24]:
print(root_dir)
midi_info = pd.read_csv(root_dir + '\Jazz-midi.csv')
# midi_info.head()

C:\Users\ahdz3\Desktop\jazzmidis


,Unnamed: 0,Name,Notes,Len_Sequence,Unique_notes,len_Uni_Notes
0,0,BreezeAndI.mid,"['E-2', 'B-2', 'B-2', 'E-3', 'B-2', 'B-2', 'E-...",427,"{'C3', 'F1', 'A2', 'B-0', 'E-1', 'E3', 'B1', '...",27
1,1,IfIHadYou.mid,"['10.2.5', '10.2.5', '2.5.8', '2.5.8', '0.3.7'...",204,"{'7.10.2', '6.9.0.2', '9.10.0.2.4', '6.9.0', '...",37
2,2,IllBeSeeingYou.mid,"['E-2', 'E-2', 'B-2', 'B-2', 'G2', 'D3', 'F2',...",455,"{'C3', 'F1', 'B-0', 'A2', 'E-1', 'D3', 'B1', '...",29
3,3,JustAGame.mid,"['B3', '6.11', 'B1', 'E4', 'B1', 'F#4', 'B1', ...",1572,"{'C3', '9.2', 'A2', 'B4', 'E3', 'A3', 'D5', 'B...",39
4,4,Unforgettable.mid,"['G2', 'G1', 'G2', 'G1', 'G2', 'G2', 'G2', 'C#...",421,"{'A0', 'C3', 'F1', 'A2', 'E-1', 'E3', 'G1', 'D...",28


In [31]:
# view which tracks are in our jazz midi files
j = 0

for msd_file_name in midi_info['Name']:

  # create path to midi file and read
  midi_path = os.path.join(data_dir, msd_file_name)
  multitrack = pypianoroll.read(midi_path)
  multitrack.set_resolution(2).pad_to_same()


  for track in multitrack.tracks:
    print(j, track.name, '\n')
    j += 1

  if j == 100:
    break
  # Piano, Guitar, Bass, Strings, Drums


0 Bass / Acoustic bass 

1 Piano / Rhodes electric 

2 Solo / Jazz guitar 

3 Strings / Slow strings 

4 Guitar / Nylon string 

5 Melody / E.Piano 6 

6 Melody / E. Piano 6 

7 Melody / diminished Rhodes 

8 Melody / diminished Rhodes 

9 Drums / Traps 

10  

11  

12  

13  

14 Bass / Acoustic bass 

15 Piano / Rhodes electric 

16 Melody / Rhodes electric 

17 Voices / Who who's 

18 Strings / Slow strings 

19 Solo / Jazz guitar 

20 Solo /(diminished) Jazz guitar 

21 Drums / Traps 

22  

23  

24  

25  

26  

27 Bass / Acoustic bass 

28 Piano / Rhodes electric 

29 Melodys /Rhodes-Mute trump 

30 Melodys /Rhodes-Mute trump 

31 Solo / Jazz guitar 

32 Strings / Slow strings 

33 Voices / Who who's 

34 Drums / Traps 

35 Bass / Acoustic bass 

36 Piano / Rhodes piano 

37 Melody / Rhodes piano 

38 Strings / Slow strings 

39 Guitar / Nylon string 

40 Solo / Jazz guitar 

41 Drums / Wire brush 

42 Love is a losing game 

43 Love is a losing game 

44 Love is a losing game

c:\Users\ahdz3\mambaforge\lib\site-packages\pretty_midi\pretty_midi.py:97: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


150 Chart Position No. 6  - 2000 

151 Remixed by Estrada 23rd January 2011 

152 Track 4 

153 Melody 

154 Track 6 

155 Track 7 

156 Track 8 

157 Track 9 

158 Track 10 

159 Track 11 

160 Track 12 

161 Track 13 

162 Track 14 

163 Track 15 

164 Track 16 

165 ZQL: Saprano Saxapho 

166 303: Synthesizer Maj 

167 303: Synthesizer Min 

168 Strings Number 2     

169 ZQL: Electric Bass   

170 ZQL: BassDrum * Shak 

171 Shaker               

172 Hall Tom Number 1    

173 Snare Drum Number 2  

174 Klak Number 2        

175 303: Hall Claves     

176 Rhodes               

177 Lead Number 8        

178 Rythm Guitar 

179 Fingered Bass 

180 Synvox 

181 Synth Brass 

182 Piano L.H. 

183 Piano R.H. 

184 Vibraphone 

185 Marimba 

186 Solo Guitar 

187 Muted Guitar 

188 Cymbals 

189 Snare 

190 Kick drums 



OSError: data byte must be in range 0..127

In [27]:
combined_pianorolls = []
i = 0

# change root_dir to training set afterwards
for msd_file_name in midi_info['Name']:

  # create path to midi file and read
  midi_path = os.path.join(data_dir, msd_file_name)
  multitrack = pypianoroll.read(midi_path)
  multitrack.set_resolution(2).pad_to_same()

  # Piano, Guitar, Bass, Strings, Drums
  # Splitting into different parts

  # RESOLVE: original code uses LPD-5 standardized pianorolls that have all 5 parts

  parts = {'piano_part': None, 'guitar_part': None, 'bass_part': None, 'strings_part': None, 'drums_part': None}
  song_length = None
  empty_array = None
  has_empty_parts = False
  for track in multitrack.tracks:
    if track.name == 'Drums':
      parts['drums_part'] = track.pianoroll
    if track.name == 'Piano':
      parts['piano_part'] = track.pianoroll
    if track.name == 'Guitar':
      parts['guitar_part'] = track.pianoroll
    if track.name == 'Bass':
      parts['bass_part'] = track.pianoroll
    if track.name == 'Strings':
      parts['strings_part'] = track.pianoroll
    if track.pianoroll.shape[0] > 0:
      empty_array = np.zeros_like(track.pianoroll)


  for k,v in parts.items():
    if v.shape[0] == 0:
      parts[k] = empty_array.copy()
      has_empty_parts = True

  # Stack all together - Piano, Guitar, Bass, Strings, Drums
  combined_pianoroll = torch.tensor([parts['piano_part'], parts['guitar_part'], parts['bass_part'], parts['strings_part'], parts['drums_part']])

  # These contain velocity information - the force with which the notes are hit - which can be standardized to 0/1 if we want (to compress)
  if has_empty_parts == False:
    combined_pianorolls.append(combined_pianoroll)
    i+=1
    print(i)

  if i == 1000:
    break

AttributeError: 'NoneType' object has no attribute 'shape'